In [ ]:
!pip install selenium requests webdriver-manager

In [ ]:
!pip show selenium requests webdriver-manager


In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from threading import Thread
import traceback

In [ ]:
def get_proxy():
    return requests.get("http://127.0.0.1:5010/get").json()

def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))
    
def Change_The_Time_Type(str_time):
    m, s = str_time.split(':')
    return int(m) * 60 + int(s)

In [ ]:
get_proxy()

In [13]:
# your spider code

def play_one(video_url):
    # ....
    retry_count = 5
    proxy = get_proxy().get("proxy")
    delete_proxy(proxy)                 # 立刻删除代理池中代理，以后要优化可以改成使用队列，保证多线程安全
    
    while retry_count > 0:
        try:
            # 设置代理
            # chromeOptions = webdriver.ChromeOptions()
            # chromeOptions.add_argument(f"-proxy-server=http：//{proxy}")
            options = Options()
            options.add_argument(f"-proxy-server=http：//{proxy}")
            options.add_argument('--no-sandbox')
            
            # 我真是大开眼界，--改成-，冒号改成中文，才能用
            # 一定要注意，=两边不能有空格，不能是这样--proxy-server = http://202.20.16.82:10152
            # browser = webdriver.Chrome(r'C:\Program Files\Google\Chrome\Application/chromedriver.exe',
            #                            options=chromeOptions)
            
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
            
            # 使用代理访问
            # 打开视频播放页
            driver.get(video_url)
            
            time.sleep(5)
            
            # 两倍速似乎有bug，b站会认为只播放了一半，取消之
            # 两倍速
            element=driver.find_element(by=By.XPATH,value="//div[@class='bpx-player-ctrl-btn bpx-player-ctrl-playbackrate']")
            webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
            element=driver.find_element(by=By.XPATH,value="//ul[@class='bpx-player-ctrl-playbackrate-menu']/li[4]")
            webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
            
            # 获取视频时长
            #element=driver.find_element_by_xpath("//ul[@class='bilibili-player-video-btn-speed-menu']/li[1]")
            #webdriver.ActionChains(driver).move_to_element(element)
            Video_Time = driver.find_element(by=By.CSS_SELECTOR, value="span.bpx-player-ctrl-time-duration").text
            print(Video_Time)
            Total_Second = Change_The_Time_Type(Video_Time)
            print(Total_Second)

            # 点击播放
           # element=driver.find_element_by_xpath("//div[@class='bpx-player-ctrl-btn bpx-player-ctrl-play']")
            # webdriver.ActionChains(driver).move_to_element(element).click(element).perform()

            # 页面最小化
            driver.minimize_window() 

            # 看完视频
            time.sleep(10) # 留出5秒余度
            
            # https://blog.csdn.net/HGGshiwo/article/details/107661135
            return
        
        except Exception as e:
            traceback.print_exc()
            retry_count -= 1
        finally:
            # 关闭页面
            driver.close()

    return None

In [14]:
def always_play(url):
    for _ in range(150):
        play_one(url)

In [15]:
thread_count = 5  # Number of threads for each URL

urls = [
    # 'https://www.bilibili.com/video/BV1Pp42117ob',
    # 'https://www.bilibili.com/video/BV1fC41157Lt',
    # 'https://www.bilibili.com/video/BV1iC411t7Ev',
    # 'https://www.bilibili.com/video/BV16H4y1H7gk',
    # 'https://www.bilibili.com/video/BV1HJ4m1j7zU',
    # 'https://www.bilibili.com/video/BV1Yx421U7VX',
    'https://www.bilibili.com/video/BV1jq421w7uo',
    'https://www.bilibili.com/video/BV1gJ4m157We',
    'https://www.bilibili.com/video/BV11M4m1X7dG'
]

In [16]:

for url in urls:
    # always_play(url)
    thread_list = [Thread(target=always_play, args=(url,)) for _ in range(thread_count)]
    
    # Start all threads for the current URL
    for t in thread_list:
        t.start()
        time.sleep(2)  # Wait before starting next thread

    # Wait for all threads for the current URL to finish
    for t in thread_list:
        t.join()

00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13


Traceback (most recent call last):
  File "/var/folders/rd/yz45hkyj3tl3yhz1wdgjn6040000gn/T/ipykernel_26688/2374874039.py", line 33, in play_one
    element=driver.find_element(by=By.XPATH,value="//div[@class='bpx-player-ctrl-btn bpx-player-ctrl-playbackrate']")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 741, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 347, in execute
    self.error_handler.check_response(response)
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/errorhandler.py", line 229, in 

00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13

13


Traceback (most recent call last):
  File "/var/folders/rd/yz45hkyj3tl3yhz1wdgjn6040000gn/T/ipykernel_26688/2374874039.py", line 43, in play_one
    Total_Second = Change_The_Time_Type(Video_Time)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rd/yz45hkyj3tl3yhz1wdgjn6040000gn/T/ipykernel_26688/86735731.py", line 8, in Change_The_Time_Type
    m, s = str_time.split(':')
    ^^^^
ValueError: not enough values to unpack (expected 2, got 1)


00:13
13


...

Traceback (most recent call last):
  File "/var/folders/rd/yz45hkyj3tl3yhz1wdgjn6040000gn/T/ipykernel_26688/2374874039.py", line 33, in play_one
    element=driver.find_element(by=By.XPATH,value="//div[@class='bpx-player-ctrl-btn bpx-player-ctrl-playbackrate']")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 741, in find_element
    return self.execute(Command.FIND_ELEMENT, {"using": by, "value": value})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/webdriver.py", line 347, in execute
    self.error_handler.check_response(response)
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/selenium/webdriver/remote/errorhandler.py", line 229, in 

00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:12
12
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:1300:13
13
00:13
13

13
00:13
13
00:13
13
00:1300:13
13
00:13
13

13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:1300:13
13
00:13
13

13
00:1300:13
13

13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:1300:13
13

13
00:13
13
00:13
13
00:13
13
00:13
13
0

Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1423, in getresponse
    response.begin()
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 331, in begin
    version, 

KeyboardInterrupt: 

Exception in thread Thread-41 (always_play):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
Exception in thread Thread-39 (always_play):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/liyuxi/Desktop/proxy_pool/env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/rd/yz45hkyj3tl3yhz1